## Preprocessing

In [1]:
import os, sys
sys.path.append(os.path.abspath("../"))
from src.data.make_dataset import get_dataset

import pandas as pd

DATASET_PATH = '../data/raw/filtered_paranmt/filtered.tsv'

# Load data
get_dataset()
df = pd.read_csv(DATASET_PATH, delimiter='\t')

Dataset already exists at /Users/hamadasalhab/Library/CloudStorage/OneDrive-АНОВОУниверситетИннополис/Disk D/Innopolis Study Materials/F23/PMLDL/Assignments/Assignment#01/text-detoxification/src/data/../../data/raw/filtered_paranmt.zip
All set. The dataset can be found in project_root_dir/data/raw.


In [2]:
df.head()

,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.785171,0.010309,0.014195,0.981983
1,1,Now you're getting nasty.,you're becoming disgusting.,0.749687,0.071429,0.065473,0.999039
2,2,"Well, we could spare your life, for one.","well, we can spare your life.",0.919051,0.268293,0.213313,0.985068
3,3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.664333,0.309524,0.053362,0.994215
4,4,I've got orders to put her down.,I have orders to kill her.,0.726639,0.181818,0.009402,0.999348


In [3]:
import pandas as pd
from collections import Counter
from itertools import chain

# Define special tokens
PAD_TOKEN = '<pad>'
SOS_TOKEN = '<sos>'
EOS_TOKEN = '<eos>'

# Build vocabularies
def build_vocab(texts, min_freq=2):
    # Count the frequencies of tokens in the texts
    counter = Counter(chain.from_iterable(texts))
    # Create the vocabulary mapping each token to a unique index
    vocab = {token: idx for idx, (token, freq) in enumerate(counter.items()) if freq >= min_freq}
    # Add special tokens to the beginning of the dictionary
    vocab = {PAD_TOKEN: 0, SOS_TOKEN: 1, EOS_TOKEN: 2, **vocab}
    return vocab

# Tokenization and numericalization
def tokenize_and_numericalize(text, vocab):
    tokens = text.split()
    numericalized = [vocab[SOS_TOKEN]] + [vocab.get(token, vocab[PAD_TOKEN]) for token in tokens] + [vocab[EOS_TOKEN]]
    return numericalized

# Tokenize texts
reference_tokenized = df['reference'].str.split().tolist()
translation_tokenized = df['translation'].str.split().tolist()

# Build vocabularies
reference_vocab = build_vocab(reference_tokenized)
translation_vocab = build_vocab(translation_tokenized)

# Numericalize texts
df['reference_numericalized'] = df['reference'].apply(lambda x: tokenize_and_numericalize(x, reference_vocab))
df['translation_numericalized'] = df['translation'].apply(lambda x: tokenize_and_numericalize(x, translation_vocab))

In [4]:
df.head()

,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox,reference_numericalized,translation_numericalized
0,0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.785171,0.010309,0.014195,0.981983,"[1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, ...","[1, 0, 1, 2, 3, 4, 3, 5, 6, 7, 8, 9, 10, 11, 1..."
1,1,Now you're getting nasty.,you're becoming disgusting.,0.749687,0.071429,0.065473,0.999039,"[1, 15, 16, 17, 18, 2]","[1, 15, 16, 17, 2]"
2,2,"Well, we could spare your life, for one.","well, we can spare your life.",0.919051,0.268293,0.213313,0.985068,"[1, 19, 20, 21, 22, 23, 24, 25, 26, 2]","[1, 18, 19, 20, 21, 22, 23, 2]"
3,3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.664333,0.309524,0.053362,0.994215,"[1, 27, 28, 29, 30, 31, 32, 33, 13, 34, 2]","[1, 24, 25, 26, 27, 28, 29, 2]"
4,4,I've got orders to put her down.,I have orders to kill her.,0.726639,0.181818,0.009402,0.999348,"[1, 35, 30, 36, 31, 37, 4, 38, 2]","[1, 30, 26, 31, 27, 32, 33, 2]"


In [5]:
# Save preprocessed data
INTERIM_DATASET_PATH = '../data/interim/preprocessed_new.tsv'
df[['reference_numericalized', 'translation_numericalized']].to_csv(INTERIM_DATASET_PATH, sep='\t', index=False)

: 